In [35]:
import numpy as np # linear algebra
import pandas as pd 
import os
xml_list=[]
img_list=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/archive/annotations'):
    for filename in filenames:
        an_path=os.path.join(dirname, filename)
        xml_list.append(an_path)
for dirname, _, filenames in os.walk('/content/drive/MyDrive/archive/images'):
    for filename in filenames:
        img_path=os.path.join(dirname, filename)
        img_list.append(img_path)       

In [21]:
len(img_list)

853

In [2]:
!pip install opencv-python

In [69]:
import glob
from xml.etree import ElementTree
import cv2
annotations_directory = '/content/drive/MyDrive/archive/annotations/'
images_directory = '/content/drive/MyDrive/archive/images/'
cropped_directory='/content/drive/MyDrive/archive/cropped/'
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
information = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'label': [], 'file': [], 'width': [], 'height': []}

for annotation in glob.glob(annotations_directory + '/*.xml'):
    print(annotation)
    tree = ElementTree.parse(annotation)
    
    for element in tree.iter():
        if 'size' in element.tag:
            for attribute in list(element):
                if 'width' in attribute.tag: 
                    width = int(round(float(attribute.text)))
                if 'height' in attribute.tag:
                    height = int(round(float(attribute.text)))    
        if 'object' in element.tag:
            for attribute in list(element):
                
                if 'name' in attribute.tag:
                    name = attribute.text                 
                    information['label'] += [name]
                    information['width'] += [width]
                    information['height'] += [height] 
                    information['file'] += [annotation.split('/')[-1][0:-4]] 
                if 'bndbox' in attribute.tag:
                    for dimension in list(attribute):
                        if 'xmin' in dimension.tag:
                            xmin = int(round(float(dimension.text)))
                            information['xmin'] += [xmin]
                        if 'ymin' in dimension.tag:
                            ymin = int(round(float(dimension.text)))
                            information['ymin'] += [ymin]                                
                        if 'xmax' in dimension.tag:
                            xmax = int(round(float(dimension.text)))
                            information['xmax'] += [xmax]                                
                        if 'ymax' in dimension.tag:
                            ymax = int(round(float(dimension.text)))
                            information['ymax'] += [ymax]

In [49]:
annotations_info_df = pd.DataFrame(information)
# Add Annotation and Image File Names
annotations_info_df['annotation_file'] = annotations_info_df['file'] + '.xml'
annotations_info_df['image_file'] = annotations_info_df['file'] + '.png'

# Tidy Grammatical Issue
annotations_info_df.loc[annotations_info_df['label'] == 'mask_weared_incorrect', 'label'] = 'mask_incorrectly_worn'

In [57]:
annotations_info_df

,xmin,ymin,xmax,ymax,label,file,width,height,annotation_file,image_file
0,166,100,243,175,with_mask,maksssksksss469,400,298,maksssksksss469.xml,maksssksksss469.png
1,254,125,315,172,with_mask,maksssksksss469,400,298,maksssksksss469.xml,maksssksksss469.png
2,353,17,381,51,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png
3,210,40,238,68,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png
4,17,72,41,97,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png
...,...,...,...,...,...,...,...,...,...,...
4067,348,8,377,54,with_mask,maksssksksss848,400,268,maksssksksss848.xml,maksssksksss848.png
4068,356,76,361,84,with_mask,maksssksksss848,400,268,maksssksksss848.xml,maksssksksss848.png
4069,84,54,199,167,with_mask,maksssksksss85,267,400,maksssksksss85.xml,maksssksksss85.png
4070,139,94,198,147,with_mask,maksssksksss852,267,400,maksssksksss852.xml,maksssksksss852.png


In [50]:
def render_image(image):
    plt.figure(figsize = (12, 8))
    plt.imshow(image)
    plt.show()
    
# Function to Convert BGR to RGB
def convert_to_RGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
annotations_info_df['image_file'].iloc[0]
image_737_path = images_directory + annotations_info_df['image_file'].iloc[0]
image_737_path
image_737 = cv2.imread(image_737_path)

render_image(convert_to_RGB(image_737))

annotation_737_path = annotations_directory + annotations_info_df['annotation_file'].iloc[0]

In [59]:
annotations_info_df['cropped_image_file'] = annotations_info_df['file']
annotations_info_df

,xmin,ymin,xmax,ymax,label,file,width,height,annotation_file,image_file,cropped_image_file
0,166,100,243,175,with_mask,maksssksksss469,400,298,maksssksksss469.xml,maksssksksss469.png,maksssksksss469
1,254,125,315,172,with_mask,maksssksksss469,400,298,maksssksksss469.xml,maksssksksss469.png,maksssksksss469
2,353,17,381,51,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png,maksssksksss476
3,210,40,238,68,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png,maksssksksss476
4,17,72,41,97,with_mask,maksssksksss476,400,267,maksssksksss476.xml,maksssksksss476.png,maksssksksss476
...,...,...,...,...,...,...,...,...,...,...,...
4067,348,8,377,54,with_mask,maksssksksss848,400,268,maksssksksss848.xml,maksssksksss848.png,maksssksksss848
4068,356,76,361,84,with_mask,maksssksksss848,400,268,maksssksksss848.xml,maksssksksss848.png,maksssksksss848
4069,84,54,199,167,with_mask,maksssksksss85,267,400,maksssksksss85.xml,maksssksksss85.png,maksssksksss85
4070,139,94,198,147,with_mask,maksssksksss852,267,400,maksssksksss852.xml,maksssksksss852.png,maksssksksss852


In [71]:
for i in range(len(annotations_info_df)):
    # Get The File Path and Read The Image
    image_filepath = images_directory + annotations_info_df['image_file'].iloc[i]
    image = cv2.imread(image_filepath)
    
    # Set The Cropped Image File Name
    annotations_info_df['cropped_image_file'].iloc[i] = annotations_info_df['cropped_image_file'].iloc[i] + '' + str(i) + '.png'
    cropped_image_filename = annotations_info_df['cropped_image_file'].iloc[i]
    
    # Get The xmin, ymin, xmax, ymax Value (Bounding Box) to Crop Image
    xmin = annotations_info_df['xmin'].iloc[i]
    ymin = annotations_info_df['ymin'].iloc[i]
    xmax = annotations_info_df['xmax'].iloc[i]
    ymax = annotations_info_df['ymax'].iloc[i]

    # Crop The Image Based on The Values Above
    cropped_image = image[ymin:ymax, xmin:xmax]
    
    # Save Cropped Image
    cropped_image_directory = os.path.join(cropped_directory, cropped_image_filename) 
    print(cropped_image_directory)
    cv2.imwrite(cropped_image_directory, cropped_image)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


/content/drive/MyDrive/archive/cropped/maksssksksss469_0.png0.png0.png0.png0.png0.png0.png
/content/drive/MyDrive/archive/cropped/maksssksksss469_1.png1.png1.png1.png1.png1.png1.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_2.png2.png2.png2.png2.png2.png2.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_3.png3.png3.png3.png3.png3.png3.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_4.png4.png4.png4.png4.png4.png4.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_5.png5.png5.png5.png5.png5.png5.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_6.png6.png6.png6.png6.png6.png6.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_7.png7.png7.png7.png7.png7.png7.png
/content/drive/MyDrive/archive/cropped/maksssksksss476_8.png8.png8.png8.png8.png8.png8.png
/content/drive/MyDrive/archive/cropped/maksssksksss47_9.png9.png9.png9.png9.png9.png9.png
/content/drive/MyDrive/archive/cropped/maksssksksss473_10.png10.png10.png10.png10.png10.png